# Parse dealii input file
This note book contains one .ipynb file as source and a phase_function_syntax.prm as input file.
Here I show how to parse the input file to a python dictionary and how to parse back from a dictionary to a deallii input file

## Key words

### 'set'

interpret things after set as (key, value).

### 'subsection'

interpret things after 'subsection' as a new dictionary and interatively call function.

### 'end'

interpret 'end' as where function returns

In [6]:
import os 
import re
import sys


################################################################################
# Parse Dealii input file into a python dictionary
################################################################################
def ParseFromDealiiInput(fin):
    inputs = {}
    line = fin.readline()
    while line is not "":
        # Skip comment lines, mark by '#' in file
        if re.match('^ *#', line):
            pass
        # Set key and value, marked by 'set' in file
        elif re.match('^.*set', line):
            temp = re.sub('^.*set ', '', line)
            temp = temp.split('=', maxsplit=1)
            key = temp[0]
            key = key.strip(' ')
            value = temp[1]
            value = re.sub('^ *', '', value)
            value = re.sub(' *(#.*)?\n$', '', value)  # BSP
            # Fix the defination of some function which
            # occupies multiple lines
            while value[-1] == '\\':
                line = fin.readline()
                line = re.sub(' *(#.*)?\n$', '', line)
                value = value + '\n' + line
            inputs[key] = value
        # Initialize new dictionary and interatively calling,
        #marked by 'subsection' in file
        elif re.match('^.*subsection', line):
            key = re.sub('^.*subsection ', '', line)
            key = key.strip('\n')
            # Fix the bug where a subsection emerges
            # multiple times
            try:
                inputs[key]
            except KeyError:
                inputs[key] = ParseFromDealiiInput(fin)
            else:
                temp = ParseFromDealiiInput(fin)
                print(iter(temp))  # debug
                inputs[key].update(temp.items())
        # Terminate and return, marked by 'end' in file
        elif re.match('^.*end', line):
            return inputs
        line = fin.readline()
    return inputs

# Open test file and parse from it
filename = "../files/phase_function_syntax.prm"
if not os.access(filename, os.R_OK):
    print("Read access not permitted on %s" % filename)
    sys.exit(1)

fin = open(filename, 'r')
inputs = ParseFromDealiiInput(fin)
print(inputs)
fin.close()

{'Dimension': '2', 'Start time': '0', 'End time': '1e15', 'Use years in output instead of seconds': 'false', 'Geometry model': {'Model name': 'box', 'Box': {'X extent': '1000000', 'Y extent': '1000000'}}, 'Gravity model': {'Model name': 'vertical', 'Vertical': {'Magnitude': '10.0'}}, 'Boundary temperature model': {'Fixed temperature boundary indicators': '3', 'List of model names': 'box', 'Box': {'Top temperature': '1000'}}, 'Boundary velocity model': {'Prescribed velocity boundary indicators': '2:function, 3:function', 'Tangential velocity boundary indicators': '0, 1', 'Function': {'Function expression': '0;-2.1422e-11', 'Variable names': 'x,y'}}, 'Initial temperature model': {'Model name': 'function', 'Function': {'Function expression': '1000.0', 'Variable names': 'x,y'}}, 'Material model': {'Model name': 'latent heat', 'Latent heat': {'Phase transition density jumps': '115.6, 200.0', 'Corresponding phase for density jump': '0, 0', 'Phase transition depths': 'all:500000|700000', 'Pha

In [7]:
###############################################################################
# Parse a python dictionary into a Dealii input file 
################################################################################
def ParseToDealiiInput(fout, outputs, layer=0):
    indent = ' ' * 4 * layer  # Indentation of output
    for key, value in outputs.items():
        if type(value) is str:
            fout.write(indent + 'set %s = %s\n' % (key, value))
        elif type(value) is dict:
            if layer == 0:
                fout.write('\n')
            fout.write(indent + 'subsection %s\n' % key)
            layer1 = layer + 1
            ParseToDealiiInput(fout, value, layer1)
            fout.write(indent + 'end\n')
            if layer == 0:
                fout.write('\n')
    return

ofilename = './test.prm'
fout = open(ofilename, 'w')
ParseToDealiiInput(fout, inputs)
fout.close()
with open(ofilename, 'r') as fin:
    print(fin.read())

set Dimension = 2
set Start time = 0
set End time = 1e15
set Use years in output instead of seconds = false

subsection Geometry model
    set Model name = box
    subsection Box
        set X extent = 1000000
        set Y extent = 1000000
    end
end


subsection Gravity model
    set Model name = vertical
    subsection Vertical
        set Magnitude = 10.0
    end
end


subsection Boundary temperature model
    set Fixed temperature boundary indicators = 3
    set List of model names = box
    subsection Box
        set Top temperature = 1000
    end
end


subsection Boundary velocity model
    set Prescribed velocity boundary indicators = 2:function, 3:function
    set Tangential velocity boundary indicators = 0, 1
    subsection Function
        set Function expression = 0;-2.1422e-11
        set Variable names = x,y
    end
end


subsection Initial temperature model
    set Model name = function
    subsection Function
        set Function expression = 1000.0
        set Variabl